In [1]:
!pip install transformers tensorflow streamlit pyngrok numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.5 MB/s eta 0:00:00


In [2]:
import streamlit as st
from transformers import TFRobertaForSequenceClassification, RobertaTokenizerFast
import tensorflow as tf
import pickle
from sklearn.preprocessing import LabelEncoder

# Load model & tokenizer
model = TFRobertaForSequenceClassification.from_pretrained("./saved_emotion_model")
tokenizer = RobertaTokenizerFast.from_pretrained("./saved_emotion_tokenizer")

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.classes_ = [
    "sadness",
    "joy",
    "love",
    "anger",
    "fear",
    "surprise"
]

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

print("Model, tokenizer, and label encoder saved successfully.")

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at ./saved_emotion_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Model, tokenizer, and label encoder saved successfully.


In [3]:
%%writefile streamlit_app.py
import streamlit as st
from transformers import TFRobertaForSequenceClassification, RobertaTokenizerFast
import tensorflow as tf
import pickle
import numpy as np

# Load model & tokenizer
model = TFRobertaForSequenceClassification.from_pretrained("./saved_emotion_model")
tokenizer = RobertaTokenizerFast.from_pretrained("./saved_emotion_tokenizer")

# Load label encoder
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Streamlit UI
st.title("Emotion Detection App")
st.write("Enter a sentence and get the predicted emotion.")

user_input = st.text_input("Type your sentence here:")

if st.button("Predict"):
    if user_input.strip() != "":
        # Tokenize input
        inputs = tokenizer(user_input, return_tensors="tf", truncation=True, padding=True, max_length=128)
        outputs = model(inputs)
        # Get prediction probabilities
        prediction = tf.nn.softmax(outputs.logits, axis=-1).numpy()[0]

        # Identify the top predicted class
        predicted_class = np.argmax(prediction)
        predicted_label = label_encoder.classes_[predicted_class]

        # Get confidence percentage
        confidence = prediction[predicted_class] * 100

        # Display result
        st.success(f"Predicted Emotion: {predicted_label} ({confidence:.2f}% confidence)")

    else:
        st.warning("Please enter a sentence.")

Writing streamlit_app.py


In [4]:
!ngrok config add-authtoken 318dey8I8ExqyGMTqpm5zcG7NDi_88uEhCXojb1snZAKF6xCs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from pyngrok import ngrok

!killall ngrok
# Run streamlit on port 8501
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app available at: {public_url}")

!streamlit run streamlit_app.py --server.port 8501

ngrok: no process found
Streamlit app available at: NgrokTunnel: "https://35bfc4a4df26.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.134.193:8501

2025-11-12 17:54:29.432660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762970069.459183    4797 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762970069.466902    4797 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762970069.487960    4797 computation_placer.cc:177] computation placer already registered. Please check linkage and a

  Stopping...
  Stopping...
